# GPP Client Design Overview

The `GPPClient` is designed to provide a starting point and modular interface for interacting with the GPP API using GraphQL. It uses the `gql` library for query execution and transport. This design abstracts common tasks such as fetching schema information, querying specific types, and mutations.

These requirements were taken from the [science requirements for GOATS for GPP](https://noirlab.atlassian.net/wiki/spaces/GOATS/pages/17592221706/GOATS+GPP+science+requirements+for+GPP+support+of+GOATS).

## Libraries Used in This Proof of Concept
- **Python**: Version `>3.10`
- **Libraries**:
  - `gql[all]`
  - `marshmallow`
  - `astropy`

### [Marshmallow](https://marshmallow.readthedocs.io/)
Marshmallow is a powerful library for object serialization and deserialization in Python. It simplifies data validation and transformation, making it easy to handle complex data structures. For this proof of concept, Marshmallow is extended to support custom fields like Astropy's `Angle`, enabling seamless integration with astronomical data.

### [gql](https://gql.readthedocs.io/)
The `gql` library provides a Pythonic way to interact with GraphQL APIs. It supports synchronous and asynchronous operations, query building with a domain-specific language (DSL), and schema introspection. This library is central to dynamically building and validating GraphQL queries in this proof of concept.


## Why Use a Library for GraphQL Queries and Transport?
- **Simplifies Query Construction**: Libraries like `gql` allow for dynamic query building, improving flexibility in interacting with GraphQL APIs without hardcoding queries.
- **Error Handling**: Provides robust error handling mechanisms and introspection capabilities to understand API structure dynamically.
- **Transport Abstraction**: Abstracts the complexity of HTTP transport, allowing for easy swapping of transport mechanisms and handling of features like authentication and timeouts.
- **Serialization and Validation**: Integration with tools like Marshmallow ensures that the API responses are validated and serialized into user-friendly formats, reducing development overhead.

## Supported Transports
The `gql` library supports a variety of transport mechanisms, including:
1. **HTTP**: The default transport for most GraphQL APIs. It supports authentication headers, timeouts, and schema fetching.
2. **WebSockets**: Enables real-time communication with GraphQL subscriptions.
3. **Batch Transport**: Allows batching multiple queries into a single HTTP request, improving efficiency.



## Project File Overview

This proof of concept includes several files to demonstrate the integration of GPP functionality with GOATS. Below is a brief description of each file:

- **`astropy_angle_field.py`**: Contains the custom Marshmallow field for handling `Angle` objects from the Astropy library, enabling serialization and deserialization of angle values.
- **`enums.py`**: Defines Enum classes for predefined values extracted from the GraphQL schema.
- **`gpp_client.py`**: A barebones GraphQL client designed to show the proof of concept for querying data using the GPP API.
- **`models.py`**: Includes Python classes used for initializing and managing unpacked data.
- **`schema.graphql`**: The GraphQL schema file downloaded from the GPP playground, serving as the foundation for query and mutation definitions in the client. Removed for privacy. To download, go to GPP playground -> SCHEMA -> DOWNLOAD -> SDL
- **`schemas.py`**: Contains Marshmallow schemas to handle data validation and serialization/deserialization for the application.

# Wishlist for the `gpp_client` Library

## Simplified Query and Mutation Handling

To streamline usage, I would like the `gpp_client` library to include prebuilt, commonly used functions that abstract the complexity of building GraphQL queries or mutations. These functions would eliminate the need for users to construct DSL queries manually, providing an intuitive interface for interacting with the API.

### Features I'd Like to See

1. **Dynamic Field Selection**
   - Ability to request all fields for a query by default.
   - Option to pass in a specific list of fields, allowing the query to be modified dynamically to include only the requested fields.

2. **Prebuilt Functions**
   - Functions like `get_observation_by_id` that encapsulate common operations (e.g., fetching an observation by ID).
   - Support for similar mutations and other queries with minimal input required from the user.

3. **Data as Classes**
   - Instead of returning raw dictionaries, the library should deserialize data into Python classes.
   - This would provide attribute-based access to data, making it easier and more intuitive to use, especially for larger or nested structures.

4. **Ease of Use**
   - Reduce the cognitive load by offering clear, reusable, and well-documented functions.
   - Maintain flexibility for power users while catering to simpler use cases for quick API interactions.

In [ ]:
from gpp_client import GPPClient
API_URL = ""
API_KEY = ""

gpp_client = GPPClient(API_URL, API_KEY, schema_path="schema.graphql")

## Automating Schema Downloads and Versioning for GPP

Notice how I include a downloaded `schema.graphql` file from the GPP playground.

In the future, schema downloads could be automated, ensuring the client always uses the latest version of the schema and stays in sync with the GraphQL API. This can be achieved through the following methods:

### Automating Schema Downloads
- **Introspection Queries**: The GraphQL API supports introspection queries to fetch the schema directly. This can be automated to regularly download and update the schema file.
- **Scripts or Tools**: CLI tools like `gql-cli` or custom scripts can be used to fetch and save the schema locally.
- **Integration with CI/CD Pipelines**: Automate schema fetching as part of build pipelines to keep the schema updated for every deployment.

### Schema Versioning
- **Version Control Systems**: Store the schema file in a version control system like Git to track changes over time.
- **Tagging Versions**: Assign version tags to schema changes to ensure compatibility with client code.
- **Backward Compatibility Checks**: Use tools or scripts to compare schema versions and ensure no breaking changes are introduced.


In [8]:
gpp_client.get_observation_by_id("o-e77")

Observation(observation_id='o-e77', existence=<Existence.PRESENT: 'PRESENT'>, title='test', subtitle=None, index=None, science_band=None, observation_time=None, instrument=None, pos_angle_constraint={'mode': <PosAngleConstraintMode.UNBOUNDED: 'UNBOUNDED'>})

In [ ]:
# Get all the fields of Observation.
gpp_client.dsl_schema.Observation._type.fields

## Extending Marshmallow Fields with Custom Astropy Angle Serialization

In this implementation, I extended the functionality of `Marshmallow`, a powerful library for object serialization and deserialization, to include support for a custom Astropy `Angle` field. This allows seamless conversion of Astropy `Angle` objects to and from serialized representations, such as JSON. 

### Why Marshmallow?

Marshmallow is a robust and flexible serialization library that provides:
- **Customizability**: The ability to define custom fields tailored to specific data types or libraries.
- **Integration**: Easy integration with various Python libraries, including `Astropy`.
- **Validation**: Built-in validation for serialized data.
- **Adoption**: Widely used in the Python community, including by the `ANTARES` team, who rely on it for their client.

## Use Case: Serializing Astropy Angles

Astropy’s `Angle` class is commonly used in astronomical calculations to represent angular measurements in various units. For example, converting an angle from arcseconds to microarcseconds can be tedious and error-prone if done manually. By creating a custom Marshmallow field, `AstropyAngleField`, we automate this process while ensuring precision and consistency.

### Key Features of `AstropyAngleField`
- **Unit Conversion**: Automatically converts angles to the specified unit during serialization.
- **Seamless Deserialization**: Reconstructs the original Astropy `Angle` object from its serialized form.
- **Integration with Marshmallow Schemas**: Easily integrates into Marshmallow schemas, allowing complex objects to include Astropy angles as fields.

In [9]:
from astropy_angle_field import AstropyAngleField
from marshmallow import Schema
from astropy import units as u
from astropy.coordinates import Angle

class TestAstropyAngle(Schema):
    test_microarcseconds = AstropyAngleField(unit=u.microarcsecond)

schema = TestAstropyAngle()
angle_value = Angle(1, unit=u.arcsecond) 
input_data = {"test_microarcseconds": angle_value}

serialized_data = schema.dump(input_data)
assert serialized_data["test_microarcseconds"] == 1000000

# GOATS Wishlist for GPP

This wishlist outlines the desired functionality for GOATS to integrate with GPP/Explore, focusing on the triggering process for ToO (Target of Opportunity) observations.

## 1. Update Target Properties
**Mutation**: `update_too_target`

### Function Signature
```python
def update_too_target(
    target_id: str, 
    new_coordinates: Tuple[float, float], 
    new_name: Optional[str] = None, 
    brightness: Optional[float] = None
) -> Any:
    """
    Update the properties of a ToO target.

    Parameters
    ----------
    target_id : str
        The unique identifier for the target.
    new_coordinates : Tuple[float, float]
        The new coordinates of the target (RA, Dec).
    new_name : Optional[str]
        The updated name for the target. Defaults to None.
    brightness : Optional[float]
        The updated brightness of the target. Defaults to None.

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```

## 2. Upload Finder Chart

**Mutation**: `upload_finder_chart`

### Function Signature
```python
def upload_finder_chart(
    observation_id: str, 
    file_path: str, 
    file_format: str
) -> Any:
    """
    Upload a finder chart for a specific observation.

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.
    file_path : str
        The path to the finder chart file.
    file_format : str
        The format of the file (e.g., 'pdf', 'png', 'jpg', 'eps').

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```

## 3. Update Observing Constraints

**Mutation**: `update_observing_constraints`

**Description**  
Users should be able to update observing constraints for a specific observation, but only within the limits set by TAC approval. For example, users can only make the constraints "worse" than approved. Dropdown menus for each field (as seen in Explore) should be mirrored in GOATS to ensure valid inputs.

**Relevant Constraints**  
Based on the Explore interface:
- **Image Quality**: Dropdown with predefined ranges (e.g., `< 0.8"`, `< 1.0"`, etc.).
- **Cloud Extinction**: Dropdown with predefined options (e.g., `< 0.3 mag`, `< 0.5 mag`, etc.).
- **Water Vapor**: Dropdown with values like `Wet`, etc.
- **Sky Background**: Dropdown with options such as `Dark`, `Bright`, etc.
- **Elevation Range**: Numeric inputs for minimum and maximum air mass.

### Using Enum Classes for Predefined Values
To enforce the use of valid predefined values, an **Enum class** could be used for fields such as `image_quality`, `cloud_extinction`, `water_vapor`, and `sky_background`.

### Function Signature
```python
from enum import Enum

class ImageQuality(Enum):
    LT_08 = "< 0.8\""
    LT_10 = "< 1.0\""
    LT_12 = "< 1.2\""

class CloudExtinction(Enum):
    LT_03 = "< 0.3 mag"
    LT_05 = "< 0.5 mag"

class WaterVapor(Enum):
    DRY = "Dry"
    WET = "Wet"

class SkyBackground(Enum):
    DARK = "Dark"
    BRIGHT = "Bright"

def update_observing_constraints(
    observation_id: str, 
    image_quality: Optional[ImageQuality] = None, 
    cloud_extinction: Optional[CloudExtinction] = None, 
    water_vapor: Optional[WaterVapor] = None, 
    sky_background: Optional[SkyBackground] = None, 
    elevation_range: Optional[dict[str, float]] = None
) -> Any:
    """
    Update the observing constraints for a specific observation.

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.
    image_quality : Optional[ImageQuality]
        The image quality constraint as an Enum (e.g., ImageQuality.LT_08).
    cloud_extinction : Optional[CloudExtinction]
        The cloud extinction constraint as an Enum (e.g., CloudExtinction.LT_03).
    water_vapor : Optional[WaterVapor]
        The water vapor constraint as an Enum (e.g., WaterVapor.DRY).
    sky_background : Optional[SkyBackground]
        The sky background constraint as an Enum (e.g., SkyBackground.BRIGHT).
    elevation_range : Optional[dict[str, float]]
        The elevation range constraint with keys "min" and "max" for the options of the dropdown.

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```
## 4. Schedule Observation

**Mutation**: `schedule_observation`

**Description**  
Users should be able to define scheduling information, including when an observation should begin and how long the scheduling window remains valid. The updated design provides more flexibility, allowing users to specify inclusion or exclusion periods, with detailed options for defining time ranges and repetition. Dates and time windows could be `datetime.datetime` or `datetime.timedelta` objects, or better, `astropy.time` objects.

**Updated Features**  
Based on the Explore interface:
- **Inclusion or Exclusion**: Users can specify whether the scheduling window includes or excludes the defined time.
- **Start Time**: A specific start time in UTC for the scheduling window.
- **Duration Options**:
  - **Forever**: No end time.
  - **Through**: A specific end time for the window.
  - **For**: A specified number of hours for the window.
- **Repetition**: Ability to repeat the window at regular intervals, specifying the period in hours when **For** is selected.

### Using Enum for Scheduling Type
To handle the inclusion/exclusion logic and duration options, **Enum classes** can be used to provide predefined values for the scheduling type and duration.


### Function Signature
```python
from enum import Enum

class SchedulingType(Enum):
    INCLUDE = "Include"
    EXCLUDE = "Exclude"

class DurationType(Enum):
    FOREVER = "Forever"
    THROUGH = "Through"
    FOR_HOURS = "For"

def schedule_observation(
    observation_id: str, 
    scheduling_type: SchedulingType, 
    start_time: str, 
    duration_type: DurationType, 
    duration_value: Optional[Union[str, int]] = None, 
    repeat_period: Optional[int] = None
) -> Any:
    """
    Pass information on scheduling and window validity for an observation.

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.
    scheduling_type : SchedulingType
        Whether the window is an inclusion or exclusion (e.g., SchedulingType.INCLUDE).
    start_time : str
        The UTC start time for the scheduling window (e.g., "2024-11-05T02:38:00").
    duration_type : DurationType
        The type of duration (e.g., DurationType.FOR_HOURS).
    duration_value : Optional[Union[str, int]]
        The value for the duration, depending on the type:
        - For DurationType.THROUGH, this is a UTC datetime string.
        - For DurationType.FOR_HOURS, this is an integer number of hours.
        - For DurationType.FOREVER, this is None.
    repeat_period : Optional[int]
        The repeat period for the scheduling window in hours. Defaults to None.

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```

## 5. Update Instrument Configuration

**Mutation**: `update_instrument_configuration`

**Description**  
Users should be able to update the instrument configuration for a specific observation. This depends on the instrument, so should be forgiving with arguments. Maybe a `dict` for configuration? 

### Function Signature
```python
from enum import Enum

class PositionAngle(Enum):
    UNCONSTRAINED = "Unconstrained"
    SPECIFIC = "Specific Angle"

class Binning(Enum):
    BIN_1X1 = "1x1"
    BIN_2X2 = "2x2"

class ReadMode(Enum):
    SLOW_LOW_GAIN = "Slow, Low Gain"
    FAST_HIGH_GAIN = "Fast, High Gain"

class ROI(Enum):
    FULL_FRAME = "Full Frame Readout"
    REGION_1 = "Region 1"

def update_instrument_configuration(
    observation_id: str, 
    configuration: dict[str, Any]
) -> Any:
    """
    Update the instrument configuration for a specific observation.

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.
    configuration : dict[str, Any]
        The updated instrument configuration. Only small changes from the
        TAC-approved configuration are allowed.

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```

## 6. Update Exposure and Observation Sequence

**Mutation**: `update_exposure_time` and `update_observation_sequence`

**Description**  
Users should be able to separately modify the **exposure time** and the **observation sequence** for a specific observation. While Explore does not currently allow editing these parameters, providing this flexibility in GOATS ensures users can:
- Adjust exposure times for individual observations.
- Reorder, add, or remove observations in the sequence.


## 7. Add Observer Notes

**Mutation**: `add_observer_notes`

### Function Signature
```python
def add_observer_notes(
    observation_id: str, 
    notes: str
) -> Any:
    """
    Pass notes to the observer for a specific observation. 

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.
    notes : str
        Specific requests or instructions for the observer.

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```

## 8. Get Observation Status

**Query**: `get_observation_status`

### Function Signature
```python
def get_observation_status(
    observation_id: str
) -> str:
    """
    Retrieve the status of a observation trigger request.

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.

    Returns
    -------
    str 
        The observation status from GPP API.
    """
```

## 9. Trigger Observation Ready

**Mutation**: `ready_observation`

### Function Signature
```python
def ready_observation(observation_id: str) -> Any:
    """
    Trigger an observation as ready.

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```

## 10. Trigger Observation Cancel

**Mutation**: `cancel_observation`

### Function Signature
```python
def cancel_observation(observation_id: str) -> Any:
    """
    Cancels an observation.

    Parameters
    ----------
    observation_id : str
        The unique identifier for the observation.

    Returns
    -------
    Any
        The response from GPP for a mutation.
    """
```

## 11. General Observation/Sequence Query Capability

We need to be able to query a wide range of information about an observation and sequence. This is one of the key strengths of GraphQL, as it allows users to fetch only the data they need while omitting unnecessary information. By supporting a flexible mechanism to pass a list of fields into queries, we can optimize bandwidth usage and reduce the volume of extraneous data.

### Desired Features
- **Field Customization**: Allow users to specify exactly which fields they want to query, tailored to their needs.
- **Dynamic Query Building**: Automatically construct queries based on the list of fields provided.